# Topic modeling
En este notebook se va a demostrar el uso de distintos modelos de extracción de temáticas (*topic modeling*) en un conjunto de textos de ejemplo sencillo.

In [1]:
import spacy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

nlp=spacy.load('en_core_web_md')

### Creación del corpus
Creamos un pequeño Corpus de ejemplo formado por 8 frases cortas. Definimos una sencilla función de normalización y aplicamos esta normalización a todo el corpus.

In [2]:
def normalize_document(doc):
    # tokenizamos el texto
    tokens = nlp(doc)
    # quitamos puntuación/espacios/stop words y cogemos el lema
    lemmas = [t.lemma_ for t in tokens if not t.is_punct and not t.is_space and not t.is_stop]
    doc = ' '.join(lemmas)
    return doc

def normalize_corpus(corpus):
    """Normaliza un corpus de documentos aplicando al función de normalización
    normalize_document() a cada documento de la lista pasada como argumento"""   
    return [normalize_document(text) for text in corpus]

toy_corpus = [
"The fox jumps over the dog",
"the fox is very clever and quick",
"The dog is slow and lazy",
"The cat is smarter than the fox and the dog",
"Python is an excellent programming language",
"Java and Ruby are other programming languages",
"Python and Java are very popular programming languages",
"Python programs are smaller than Java programs"]

norm_corpus = normalize_corpus(toy_corpus)
norm_corpus

['fox jump dog',
 'fox clever quick',
 'dog slow lazy',
 'cat smart fox dog',
 'Python excellent programming language',
 'Java Ruby programming language',
 'Python Java popular programming language',
 'Python program small Java program']

## Topic modeling usando Scikit-learn
La librería `scikit-learn` implementa los modelos *Latent Semantic Analysis* (LSA) y *Latent Dirichlet Allocation* (LDA).  
Partimos de un modelo TF-IDF para el modelado LSA y de un modelo BoW para el modelado LDA

### Modelo LSA

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# usamos características tf-idf para LSA.
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(norm_corpus)

In [4]:
tfidf

<8x18 sparse matrix of type '<class 'numpy.float64'>'
	with 30 stored elements in Compressed Sparse Row format>

In [5]:
tfidf_vectorizer.get_feature_names_out()

array(['cat', 'clever', 'dog', 'excellent', 'fox', 'java', 'jump',
       'language', 'lazy', 'popular', 'program', 'programming', 'python',
       'quick', 'ruby', 'slow', 'small', 'smart'], dtype=object)

Definimos una función de ayuda para mostrar los resultados (términos asociados a cada tema)

In [6]:
def print_top_words(model, feature_names, n_top_words):
    """Función auxiliar para mostrar los términos más importantes
    de cada topic"""
    for topic_idx, topic in enumerate(model.components_):
        message = f"Topic #{topic_idx}: "
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

Calculamos los modelos para nuestro corpus (método `fit`) y vemos cuáles son los 3 términos con más peso para cada *topic*. Cada modelo asigna un grado de pertenencia en cada tema a cada término del vocabulario de la matriz tfidf o bow utilizada como entrada.

In [7]:
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation

# Ajustamos el modelo LSA
lsa = TruncatedSVD(n_components=2).fit(tfidf)

print("\nTopics en modelo LSA:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print_top_words(lsa, tfidf_feature_names, 2)


Topics en modelo LSA:
Topic #0: programming language
Topic #1: dog fox



In [8]:
lsa

TruncatedSVD()

El método `fit` aprende la matriz de `topics` x `términos` para el corpus dado

In [9]:
lsa.components_.shape

(2, 18)

In [10]:
pd.DataFrame(np.round(lsa.components_, 4), columns=tfidf_feature_names)

,cat,clever,dog,excellent,fox,java,jump,language,lazy,popular,program,programming,python,quick,ruby,slow,small,smart
0,-0.0000,-0.0000,-0.0000,0.2303,-0.0000,0.3901,-0.000,0.5035,-0.0000,0.2357,0.1469,0.5035,0.3901,-0.0000,0.2303,-0.0000,0.0735,-0.0000
1,0.2596,0.1808,0.5586,0.0000,0.5586,0.0000,0.332,0.0000,0.1808,0.0000,0.0000,0.0000,0.0000,0.1808,0.0000,0.1808,0.0000,0.2596


In [11]:

np.round(tfidf.todense(), 4)

array([[0.    , 0.    , 0.5056, 0.    , 0.5056, 0.    , 0.6991, 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.    , 0.6296, 0.    , 0.    , 0.4553, 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.6296, 0.    , 0.    ,
        0.    , 0.    ],
       [0.    , 0.    , 0.4553, 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.6296, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.6296,
        0.    , 0.    ],
       [0.573 , 0.    , 0.4144, 0.    , 0.4144, 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.573 ],
       [0.    , 0.    , 0.    , 0.6239, 0.    , 0.    , 0.    , 0.4512,
        0.    , 0.    , 0.    , 0.4512, 0.4512, 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.4512, 0.    , 0.4512,
        0.    , 0.    , 0.    , 0.4512, 0.    , 0.    , 0.6239, 0.    ,
        0. 

Podemos ver el porcentaje de pertenencia a cada *topic* de cada una de los documentos asignados por el modelo con el método `transform`:

In [12]:
lsa.transform(tfidf)

array([[-8.72755885e-16,  7.96994363e-01],
       [-5.34931207e-16,  4.82026015e-01],
       [-9.25831353e-16,  4.82026015e-01],
       [-8.91759086e-16,  7.60503821e-01],
       [ 7.74056505e-01,  9.73592307e-16],
       [ 7.74056505e-01,  7.56282967e-16],
       [ 8.69066155e-01,  9.35429431e-16],
       [ 3.78882291e-01,  6.24296431e-16]])

In [13]:
np.round(lsa.transform(tfidf), 4)

array([[-0.    ,  0.797 ],
       [-0.    ,  0.482 ],
       [-0.    ,  0.482 ],
       [-0.    ,  0.7605],
       [ 0.7741,  0.    ],
       [ 0.7741,  0.    ],
       [ 0.8691,  0.    ],
       [ 0.3789,  0.    ]])

Cada fila corresponde a un documento del Corpus, y cada columna el grado de pertenencia a ese tema del documento.  
El modelo ha separado correctamente el corpus en las dos temáticas principales:

In [14]:
np.argmax(lsa.transform(tfidf), axis=1)

array([1, 1, 1, 1, 0, 0, 0, 0])

### Modelo LDA

In [15]:
# usamos características BoW para LDA.
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(norm_corpus)

In [16]:
tf

<8x18 sparse matrix of type '<class 'numpy.int64'>'
	with 30 stored elements in Compressed Sparse Row format>

In [17]:
# Ajustamos el modelo LDA
lda = LatentDirichletAllocation(n_components=2, max_iter=5,
                                learning_method='batch',
                                learning_offset=50.,
                                random_state=0).fit(tf)

print("\nTopics en modelo LDA:")
tf_feature_names = tf_vectorizer.get_feature_names_out()
print_top_words(lda, tf_feature_names, 2)


Topics en modelo LDA:
Topic #0: fox dog
Topic #1: programming language



El atributo `components_` contiene los parámetros de la distribución de términos en *topics*.

In [18]:
lda.components_.shape

(2, 18)

In [19]:
pd.DataFrame(lda.components_, columns=tfidf_feature_names)

,cat,clever,dog,excellent,fox,java,jump,language,lazy,popular,program,programming,python,quick,ruby,slow,small,smart
0,1.494703,1.493142,2.929819,0.505725,3.493344,1.334297,1.492834,0.505621,0.539143,0.504793,2.484115,0.50562,1.322094,1.493142,0.505889,0.538785,1.48459,1.494703
1,0.505297,0.506858,1.070181,1.494275,0.506656,2.665703,0.507166,3.494379,1.460857,1.495207,0.515885,3.49438,2.677906,0.506858,1.494111,1.461215,0.51541,0.505297


Normalizando esta matriz muestra la distribución de términos dentro de cada *topic*

In [20]:
distribucion = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
pd.DataFrame(distribucion, columns=tfidf_feature_names)

,cat,clever,dog,excellent,fox,java,jump,language,lazy,popular,program,programming,python,quick,ruby,slow,small,smart
0,0.061963,0.061899,0.121457,0.020965,0.144818,0.055314,0.061886,0.020961,0.022350,0.020926,0.102980,0.020961,0.054808,0.061899,0.020972,0.022335,0.061544,0.061963
1,0.020311,0.020374,0.043018,0.060065,0.020366,0.107153,0.020386,0.140463,0.058722,0.060102,0.020737,0.140463,0.107643,0.020374,0.060058,0.058736,0.020718,0.020311


Como en el caso del LSA, podemos ver el grado de pertenencia de cada documento a cada *topic*

In [21]:
lda.transform(tf)

array([[0.86941004, 0.13058996],
       [0.870979  , 0.129021  ],
       [0.28288661, 0.71711339],
       [0.89559705, 0.10440295],
       [0.10511141, 0.89488859],
       [0.10518116, 0.89481884],
       [0.08917537, 0.91082463],
       [0.83234111, 0.16765889]])

In [22]:
lda.transform(tf).shape

(8, 2)

In [23]:
np.sum(lda.transform(tf), axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [24]:
np.argmax(lda.transform(tf), axis=1)

array([0, 0, 1, 0, 1, 1, 1, 0])

Probar Textos nuevos

In [51]:
textos = ['My friend fox is programming python', 'python is a kind of snake']
textos_norm = normalize_corpus(textos)

In [52]:
textos_norm

['friend fox program python', 'python kind snake']

In [53]:
textos_tfidf = tfidf_vectorizer.transform(textos_norm)
textos_tfidf

<2x18 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [54]:
lsa.transform(textos_tfidf)

array([[2.99969036e-01, 2.82444256e-01],
       [3.90107400e-01, 5.40358719e-16]])

### Modelo NMF

In [25]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=2, init='random', random_state=42)
nmf.fit(tfidf)

NMF(init='random', n_components=2, random_state=42)

In [26]:
print("\nTopics en modelo NMF:")
print_top_words(nmf, tfidf_feature_names, 2)


Topics en modelo NMF:
Topic #0: programming language
Topic #1: dog fox



In [27]:
nmf.components_.shape

(2, 18)

In [28]:
pd.DataFrame(nmf.components_, columns=tfidf_feature_names)

,cat,clever,dog,excellent,fox,java,jump,language,lazy,popular,program,programming,python,quick,ruby,slow,small,smart
0,0.000000,0.000000,0.000000,0.35426,0.000000,6.001331e-01,0.000000,0.774588,0.00000,0.362547,0.226063,0.774588,6.001331e-01,0.000000,0.35426,0.00000,1.130316e-01,0.000000
1,0.336995,0.234636,0.725113,0.00000,0.725031,1.967075e-07,0.430912,0.000000,0.23475,0.000000,0.000001,0.000000,1.967075e-07,0.234636,0.00000,0.23475,5.093429e-07,0.336995


In [29]:
#calculamos para nuestro corpus las temáticas
nmf.transform(tfidf)

array([[0.00000000e+00, 6.14054550e-01],
       [0.00000000e+00, 3.71329882e-01],
       [0.00000000e+00, 3.71436925e-01],
       [0.00000000e+00, 5.85939850e-01],
       [5.03163521e-01, 0.00000000e+00],
       [5.03163520e-01, 0.00000000e+00],
       [5.64922946e-01, 0.00000000e+00],
       [2.46285794e-01, 6.06906318e-07]])

Esta matriz es el producto vectorial de la matriz TFIDF por la transpuesta de los componentes del NMF:

In [30]:
tfidf @ nmf.components_.T

array([[0.00000000e+00, 1.03443550e+00],
       [0.00000000e+00, 6.25541840e-01],
       [0.00000000e+00, 6.25722164e-01],
       [0.00000000e+00, 9.87073514e-01],
       [1.19079349e+00, 8.87545929e-08],
       [1.19079349e+00, 8.87545930e-08],
       [1.33695417e+00, 1.61801646e-07],
       [5.82863244e-01, 1.15143765e-06]])

## Topic modeling usando librería Gensim
La librería `gensim` implementa los siguientes modelos:  
* [Latent Semantic Indexing, LSI (or sometimes LSA)](https://en.wikipedia.org/wiki/Latent_semantic_indexing) transforms documents from either bag-of-words or (preferrably) TfIdf-weighted space into a latent space of a lower dimensionality.  
* [Latent Dirichlet Allocation, LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is yet another transformation from bag-of-words counts into a topic space of lower dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA), so LDA’s topics can be interpreted as probability distributions over words. These distributions are, just like with LSA, inferred automatically from a training corpus. Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).  
* [Hierarchical Dirichlet Process, HDP](http://jmlr.csail.mit.edu/proceedings/papers/v15/wang11a/wang11a.pdf) is a non-parametric bayesian method (note the missing number of requested topics.

La entrada a los modelos de `gensim`
 debe ser una lista de tokens y no un texto por cada documento del corpus, por lo que hay que cambiar la función de normalización.

In [31]:
def normalize_tokenize_document(doc):
    # tokenizamos el texto
    tokens = nlp(doc)
    # quitamos puntuación/espacios y cogemos el lema
    lemmas = [t.lemma_.lower() for t in tokens if not t.is_punct and not t.is_space and not t.is_stop]
    return lemmas

def normalize_tokenize_corpus(corpus):
    """Normaliza un corpus de documentos aplicando al función de normalización
    normalize_tokenize_document() a cada documento de la lista pasada como argumento"""   
    return [normalize_tokenize_document(text) for text in corpus]
        
norm_tokenized_corpus = normalize_tokenize_corpus(toy_corpus)
norm_tokenized_corpus

[['fox', 'jump', 'dog'],
 ['fox', 'clever', 'quick'],
 ['dog', 'slow', 'lazy'],
 ['cat', 'smart', 'fox', 'dog'],
 ['python', 'excellent', 'programming', 'language'],
 ['java', 'ruby', 'programming', 'language'],
 ['python', 'java', 'popular', 'programming', 'language'],
 ['python', 'program', 'small', 'java', 'program']]

In [32]:
norm_corpus

['fox jump dog',
 'fox clever quick',
 'dog slow lazy',
 'cat smart fox dog',
 'Python excellent programming language',
 'Java Ruby programming language',
 'Python Java popular programming language',
 'Python program small Java program']

Al igual que en los modelos de la librería `scikit-learn`, primero generamos matrices de características BoW y TF-IDF como paso previo a aplicar los modelos de topic-modeling.  
En `gensim` estas matrices se calculan de manera diferente a `scikit-learn`

In [33]:
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel, TfidfModel

#diccionario de términos únicos del corpus
dictionary = Dictionary(norm_tokenized_corpus)
#creamos matriz BoW
corpus_bow = [dictionary.doc2bow(text)
                 for text in norm_tokenized_corpus]
#creamos matriz TF-IDF del corpus a partir de BoW
tfidf = TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

In [34]:
corpus_bow[0]

[(0, 1), (1, 1), (2, 1)]

In [35]:
corpus_tfidf[0]

[(0, 0.39239043318859274), (1, 0.39239043318859274), (2, 0.8319011334792957)]

In [36]:
corpus_tfidf

In [37]:
len(dictionary.items())

18

Cada tupla indica la frecuencia del ítem *i* en el documento, donde *i* es el índice de las palabras en el vocabulario del diccionario de Gensim.

In [38]:
[(i, k) for i,k in dictionary.items()]

[(0, 'dog'),
 (1, 'fox'),
 (2, 'jump'),
 (3, 'clever'),
 (4, 'quick'),
 (5, 'lazy'),
 (6, 'slow'),
 (7, 'cat'),
 (8, 'smart'),
 (9, 'excellent'),
 (10, 'language'),
 (11, 'programming'),
 (12, 'python'),
 (13, 'java'),
 (14, 'ruby'),
 (15, 'popular'),
 (16, 'program'),
 (17, 'small')]

### Latent Semantic Indexing
Los modelos de *topic modeling* de `gensim` asignan un peso de pertenencia de cada término del diccionario bow/tfidf a cada tema:

In [39]:
lsi = LsiModel(corpus_tfidf, 
                      id2word=dictionary,
                      num_topics=2)

for index, topic in lsi.print_topics(2):
    print(f'Topic #{str(index)}\n{topic}\n')

Topic #0
-0.459*"language" + -0.459*"programming" + -0.344*"java" + -0.344*"python" + -0.336*"popular" + -0.318*"ruby" + -0.318*"excellent" + -0.148*"program" + -0.074*"small" + 0.000*"jump"

Topic #1
-0.459*"dog" + -0.459*"fox" + -0.444*"jump" + -0.322*"smart" + -0.322*"cat" + -0.208*"slow" + -0.208*"lazy" + -0.208*"quick" + -0.208*"clever" + -0.000*"popular"



In [40]:
lsi.get_topics().shape

(2, 18)

La matriz LSI generada es un objeto específico de Gensim que funciona como un *iterable*. Contiene el grado de pertenencia al *topic* o *topics* más representativos del documento.

In [41]:
topics_lsi = lsi[corpus_tfidf]
topics_lsi

In [42]:
topics_lsi[0]

[(1, -0.7296053406305381)]

In [43]:
for t in topics_lsi:
    print(t)

[(1, -0.7296053406305381)]
[(1, -0.42469706069369884)]
[(1, -0.4246970606936994)]
[(1, -0.6892950729735758)]
[(0, -0.7070677687653235)]
[(0, -0.7070677687653237)]
[(0, -0.7950457769187111)]
[(0, -0.29788705652128333)]


El objeto *TransformedCorpus* sólo muestra los componentes distintos de cero de la matriz LSI del corpus:

In [44]:
import gensim
gensim.matutils.corpus2dense(lsi[corpus_tfidf], lsi.num_topics).T 

array([[ 0.        , -0.7296053 ],
       [ 0.        , -0.42469707],
       [ 0.        , -0.42469707],
       [ 0.        , -0.68929505],
       [-0.7070678 ,  0.        ],
       [-0.7070678 ,  0.        ],
       [-0.7950458 ,  0.        ],
       [-0.29788706,  0.        ]], dtype=float32)

### Latent Dirichlet Allocation

In [45]:
lda = LdaModel(corpus_bow, 
                      id2word=dictionary,
                      iterations=1000,
                      num_topics=2)
for index, topic in lda.print_topics(2):
    print('Topic #{}\n{}\n'.format(str(index), topic))

Topic #0
0.098*"fox" + 0.096*"python" + 0.095*"language" + 0.090*"programming" + 0.075*"dog" + 0.060*"java" + 0.058*"popular" + 0.056*"quick" + 0.055*"clever" + 0.049*"excellent"

Topic #1
0.103*"java" + 0.091*"program" + 0.088*"dog" + 0.074*"programming" + 0.069*"language" + 0.067*"python" + 0.066*"fox" + 0.056*"small" + 0.056*"ruby" + 0.048*"smart"



In [46]:
topics_lda = lda[corpus_bow]
topics_lda

In [47]:
topics_lda[0]

[(0, 0.64316183), (1, 0.35683814)]

In [48]:
for t in topics_lda:
    print(t)

[(0, 0.6430716), (1, 0.35692844)]
[(0, 0.85649794), (1, 0.14350207)]
[(0, 0.18027571), (1, 0.81972426)]
[(0, 0.43494013), (1, 0.56505984)]
[(0, 0.86306965), (1, 0.13693038)]
[(0, 0.17954321), (1, 0.8204568)]
[(0, 0.8557903), (1, 0.14420965)]
[(0, 0.103704154), (1, 0.89629585)]


In [49]:
lda.get_topics().shape

(2, 18)

In [50]:
gensim.matutils.corpus2dense(lda[corpus_bow], lda.num_topics).T #en forma de matriz

array([[0.6430459 , 0.35695413],
       [0.856462  , 0.14353801],
       [0.18026392, 0.81973606],
       [0.4349884 , 0.5650116 ],
       [0.86306804, 0.13693202],
       [0.17955117, 0.8204489 ],
       [0.8556897 , 0.14431024],
       [0.10369834, 0.8963016 ]], dtype=float32)

### Hierarchical Dirichlet Process

In [ ]:
#no hay que especificar un núm. de topics
hdp = HdpModel(corpus_bow, 
                      id2word=dictionary)
for index, topic in hdp.print_topics(2):
    print('Topic #{}\n{}\n'.format(str(index), topic))

In [ ]:
for index, topic in hdp.print_topics(8):
    print('Topic #{}\n{}\n'.format(str(index), topic))

In [ ]:
topics_hdp = hdp[corpus_bow]
topics_hdp

In [ ]:
for t in topics_hdp:
    print(t)

In [ ]:
np.round(
    gensim.matutils.corpus2dense(hdp[corpus_bow], hdp.m_num_docs_processed).T, 4) #en forma de matriz

### Estimación de temática principal
Podemos calcular la pertenencia de cada documento a una temática a partir de su modelo calculado:  
El modelo LSI sólo devuelve el grado de pertencia de cada documento a los *topics* más relevantes a ese documento.

In [ ]:
corpus_lsi = lsi[corpus_tfidf]
for i, doc in enumerate(corpus_lsi):
     print(doc, toy_corpus[i])

Sin embargo LDA y HDP calculan la pertenencia a cada tema y devuelven una lista de tuplas por cada tema (nº de tema, grado de pertenencia).

In [ ]:
corpus_lda = lda[corpus_bow]
for i, doc in enumerate(corpus_lda):
     print(doc, toy_corpus[i])

En el modelo LDA el peso de cada palabra en cada *topic* es una probabilidad

In [ ]:
lda.get_topics()

In [ ]:
lda.get_topics().shape

In [ ]:
np.sum(lda.get_topics(), axis=1)

Y las pertenencias de un documento a cada *topic* también son un valor de probabilidad

In [ ]:
gensim.matutils.corpus2dense(lda[corpus_bow], lda.num_topics).T

In [ ]:
np.sum(gensim.matutils.corpus2dense(lda[corpus_bow], lda.num_topics).T, axis=1)

In [ ]:
np.argmax(gensim.matutils.corpus2dense(lda[corpus_bow], lda.num_topics).T, axis=1)

Con el modelo HDP no se especifica un número de temas sino que se definen automáticamente (con importancia decreciente)

In [ ]:
# Solución
corpus_hdp = hdp[corpus_bow]
for i, doc in enumerate(corpus_hdp):
     print(doc, toy_corpus[i])

In [ ]:
np.argmax(gensim.matutils.corpus2dense(hdp[corpus_bow], len(corpus_bow)).T, axis=1)

In [ ]:
norm_corpus

Podemos obtener los términos relevantes para cada tema y su importancia con el método `show_topics` del modelo:

In [ ]:
lsitopics = [[(word,prob) for word, prob in topic] for topicid, topic in lsi.show_topics(formatted=False)]

hdptopics = [[(word,prob) for word, prob in topic] for topicid, topic in hdp.show_topics(formatted=False)]

ldatopics = [[(word,prob) for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]

In [ ]:
ldatopics

In [ ]:
lsitopics

### Topic Coherence
La librería `gensim` proporciona una funcionalidad para identificar qué modelo de *topic modeling* se adapta mejor al corpus. La función `CoherenceModel` calcula una puntuación sobre la coherencia del modelo, que podemos usar para compararlos. Esta función utiliza las palabras que definen cada tópico en los modelos.

In [ ]:
lsitopics = [[word for word, prob in topic] for topicid, topic in lsi.show_topics(formatted=False)]

hdptopics = [[word for word, prob in topic] for topicid, topic in hdp.show_topics(formatted=False)]

ldatopics = [[word for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]



In [ ]:
lsitopics

In [ ]:
lsi_coherence = CoherenceModel(topics=lsitopics[:10], texts=norm_tokenized_corpus,
                               dictionary=dictionary, window_size=10).get_coherence()

hdp_coherence = CoherenceModel(topics=hdptopics[:10], texts=norm_tokenized_corpus, 
                               dictionary=dictionary, window_size=10).get_coherence()

lda_coherence = CoherenceModel(topics=ldatopics, texts=norm_tokenized_corpus,
                               dictionary=dictionary, window_size=10).get_coherence()

In [ ]:
lsi_coherence

In [ ]:
def evaluate_bar_graph(coherences, indices):
    """
    Función para dibujar una gráfica de barras con:
    
    coherences: lista de los valores de coherencia
    indices: textos para etiquetar las barras.
    Ambos parámetros deben tener la misma longitud
    """
    assert len(coherences) == len(indices)
    n = len(coherences)
    x = np.arange(n)
    plt.bar(x, coherences, width=0.2, tick_label=indices, align='center')
    plt.xlabel('Modelos')
    plt.ylabel('Valor Coherencia')

In [ ]:
evaluate_bar_graph([lsi_coherence, hdp_coherence, lda_coherence],
                   ['LSI','HDP', 'LDA'])